In [125]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow import keras
from keras import layers
import tensorflow as tf

In [126]:
# loading and preprocessing data
df = pd.read_excel('tmp.xlsx', header=1,)

# removing the ruble symbol and non-printing characters
df['Loan issued'] = pd.to_numeric(df['Loan issued'].str.replace('₽', '').str.replace('\u00A0', ''))
df['Earned interest'] = pd.to_numeric(df['Earned interest'].str.replace('₽', '').str.replace('\u00A0', ''))
df['Unpaid,  full amount'] = pd.to_numeric(df['Unpaid,  full amount'].str.replace('₽', '').str.replace('\u00A0', ''))

# casting some columns to percentage format
df['Comission, %'] = df['Comission, %'] * 100
df['EL'] = df['EL'] * 100
df = df.drop(columns=['Unnamed: 6'], axis=1)

# column definition
df = df[['Comission, %', 'Rating', 'Loan issued', 'Earned interest', 'Unpaid,  full amount', 'EL']]

In [127]:
# calculation of required values
df['loss'] = df['Unpaid,  full amount'] * df['EL']
df['InvestorProfit'] = df['Earned interest'] - df['loss']

df['Commission'] = df['Loan issued'] * df['Comission, %'] / 100  # calculation of the absolute value of the commission

df['Profit'] = df['InvestorProfit'] + df['Commission']
df['Profit%'] = df['Profit'] / df['Loan issued']


In [128]:
# defining labels for classification

label1 = np.where(df['Profit%'] > 0)
df['label'] = 0
df['label'].iloc[label1] = 1

/var/folders/bg/wk91sfp51yjcs62nkzhg61k00000gn/T/ipykernel_38157/35014661.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'].iloc[label1] = 1


In [129]:
# Dataset making
X, Y_reg, Y_class = [], [], []

for i in df.index:
    X.append(df[['Comission, %', 'Rating', 'Loan issued', 'Earned interest', 'Unpaid,  full amount', 'EL']].iloc[i].values)
    Y_reg.append(df['Profit%'][i])
    Y_class.append(df['label'][i])
    

In [130]:
# Data scaling
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# Regression. Forecasting expected profit value.

In [131]:
# Slicing original dataset to train and test samples
X_train, X_test, y_train, y_test = train_test_split(X_scaled, Y_reg, test_size=0.2, random_state=42)

### Neural network (CNN for regression)

In [132]:
# Defining model
inputs = keras.Input(shape=(6,1))
x = layers.BatchNormalization()(inputs)
x = layers.Conv1D(filters=20, kernel_size=(2), activation='relu')(x)
x = layers.Conv1D(filters=15, kernel_size=(2), activation='relu')(x)
x = layers.Conv1D(filters=10, kernel_size=(1), activation='relu')(x)
x = layers.Dense(250, activation='relu')(x)
x = layers.Dense(200, activation='relu')(x)
x = layers.Flatten()(x)
x = layers.Dropout(0.2)(x)
outputs = layers.Dense(1, activation='linear')(x)

model = keras.Model(inputs=inputs, outputs=outputs, name="regression-model")
model.summary()

Model: "regression-model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_24 (InputLayer)       [(None, 6, 1)]            0         
                                                                 
 batch_normalization_23 (Ba  (None, 6, 1)              4         
 tchNormalization)                                               
                                                                 
 conv1d_19 (Conv1D)          (None, 5, 20)             60        
                                                                 
 conv1d_20 (Conv1D)          (None, 4, 15)             615       
                                                                 
 conv1d_21 (Conv1D)          (None, 4, 10)             160       
                                                                 
 dense_90 (Dense)            (None, 4, 250)            2750      
                                                  

In [133]:
model.compile(loss=tf.keras.losses.LogCosh(),
              optimizer=tf.keras.optimizers.legacy.Adam(0.001),
              metrics=['mse', 'mae',])

In [134]:
# Model training
ephs = 28

history = model.fit(X_train, np.array(y_train), batch_size=8 , epochs=ephs, validation_split=0.3)
test_scores = model.evaluate(X_test, np.array(y_test), verbose=1)

print("Test loss:", test_scores[0])
print("Test mse:", test_scores[1])
print("Test mae:", test_scores[2])

Epoch 1/28
38/38 [==============================] - 0s 3ms/step - loss: 2.7909 - mse: 111.0922 - mae: 3.1925 - val_loss: 2.7960 - val_mse: 114.8325 - val_mae: 3.1638
Epoch 2/28
38/38 [==============================] - 0s 2ms/step - loss: 2.7356 - mse: 110.0243 - mae: 3.1078 - val_loss: 2.7990 - val_mse: 115.0382 - val_mae: 3.1618
Epoch 3/28
38/38 [==============================] - 0s 2ms/step - loss: 2.6963 - mse: 109.4187 - mae: 3.0132 - val_loss: 2.7280 - val_mse: 113.3426 - val_mae: 3.0459
Epoch 4/28
38/38 [==============================] - 0s 2ms/step - loss: 2.6586 - mse: 107.5881 - mae: 2.9533 - val_loss: 2.7002 - val_mse: 110.3477 - val_mae: 3.0724
Epoch 5/28
38/38 [==============================] - 0s 2ms/step - loss: 2.5829 - mse: 101.7825 - mae: 2.9117 - val_loss: 2.6399 - val_mse: 102.8648 - val_mae: 3.0883
Epoch 6/28
38/38 [==============================] - 0s 2ms/step - loss: 2.4379 - mse: 90.4710 - mae: 2.7950 - val_loss: 2.2525 - val_mse: 80.6259 - val_mae: 2.6279
Epoch 

### Classic ML models (regression)

In [135]:
# Testing classic ML algorithms 

from sklearn import linear_model
reg_Lasso = linear_model.Lasso(alpha=0.1)
reg_Lasso.fit(X_train,y_train)

from sklearn.linear_model import ARDRegression
reg_ard = ARDRegression()
reg_ard.fit(X_train, y_train)

from sklearn.linear_model import ElasticNet
reg_enet = ElasticNet(alpha=0.08, l1_ratio=0.5)
reg_enet.fit(X_train, y_train)


ElasticNet(alpha=0.08)

In [136]:
# Making a forecast using the resulting models 
predicted_y_Lasso = reg_Lasso.predict(X_test)
predicted_y_ard = reg_ard.predict(X_test)
predicted_y_enet = reg_enet.predict(X_test)

In [137]:
# Receiving model metrics

from sklearn.metrics import explained_variance_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

EVS_Lasso = explained_variance_score(y_test, predicted_y_Lasso)
MAE_Lasso = mean_absolute_error(y_test, predicted_y_Lasso)
MSE_Lasso = mean_squared_error(y_test, predicted_y_Lasso)

EVS_ard = explained_variance_score(y_test, predicted_y_ard)
MAE_ard = mean_absolute_error(y_test, predicted_y_ard)
MSE_ard = mean_squared_error(y_test, predicted_y_ard)

EVS_enet = explained_variance_score(y_test, predicted_y_enet)
MAE_enet = mean_absolute_error(y_test, predicted_y_enet)
MSE_enet = mean_squared_error(y_test, predicted_y_enet)

### Regression results

In [138]:
# Returns summary table for gotten model metrics

reg_summary = pd.DataFrame({'Lasso':[EVS_Lasso, MAE_Lasso, MSE_Lasso],
                            'ARDRegression':[EVS_ard, MAE_ard, MSE_ard],
                            'ElasticNet':[EVS_enet, MAE_enet, MSE_enet],
                            'CNN':[None, test_scores[2], test_scores[1]]},
                           index=['EVS', 'MAE', 'MSE']).T
reg_summary

,EVS,MAE,MSE
Lasso,0.778256,1.803692,43.906529
ARDRegression,0.835284,1.827691,32.814800
ElasticNet,0.375667,3.724602,123.186156
CNN,NaN,1.241022,16.875847


Вывод: для классификации предлагается использовать модели глубокого обучения,
т.к. они показывают лучшие результаты. В частности, сверточную нейронную сеть.
Каждую полученную модель можно сохранить и использовать в дальнейшем.

# Classification. Predicts 1 when profit > 0, and 0 otherwise.

In [153]:
# Slicing original dataset to train and test samples
X_train, X_test, y_train, y_test = train_test_split(X_scaled, Y_class, test_size=0.2, random_state=42)

### Neural networks

Convolutional Neural network

In [140]:
# Defining model
inputs = keras.Input(shape=(6,1))
x = layers.BatchNormalization()(inputs)

x = layers.Conv1D(filters=5, kernel_size=(2), activation='relu')(x)
x = layers.Dense(300, activation='relu')(x)
x = layers.Dense(150, activation='relu')(x)
x = layers.Dense(75, activation='relu')(x)
x = layers.Dropout(0.2)(x)
x = layers.Flatten()(x)
outputs = keras.layers.Dense(1, activation='sigmoid')(x)

model_class = keras.Model(inputs=inputs, outputs=outputs, name="classification-model")
model_class.summary()

Model: "classification-model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_25 (InputLayer)       [(None, 6, 1)]            0         
                                                                 
 batch_normalization_24 (Ba  (None, 6, 1)              4         
 tchNormalization)                                               
                                                                 
 conv1d_22 (Conv1D)          (None, 5, 5)              15        
                                                                 
 dense_93 (Dense)            (None, 5, 300)            1800      
                                                                 
 dense_94 (Dense)            (None, 5, 150)            45150     
                                                                 
 dense_95 (Dense)            (None, 5, 75)             11325     
                                              

In [141]:
model_class.compile(
    loss=tf.keras.losses.MeanSquaredError(),
    optimizer=keras.optimizers.legacy.Adam(learning_rate=0.002),
    metrics=[ 'accuracy', tf.keras.metrics.Precision(),tf.keras.metrics.Recall()],
)

In [142]:
# Model training
ephs = 16

history = model_class.fit(X_train, np.array(y_train), batch_size=8 , epochs=ephs, validation_split=0.2)
test_scores_CNN_class = model_class.evaluate(X_test, np.array(y_test), verbose=1)


Epoch 1/16
43/43 [==============================] - 0s 4ms/step - loss: 0.1373 - accuracy: 0.8676 - precision_21: 0.0000e+00 - recall_21: 0.0000e+00 - val_loss: 0.1257 - val_accuracy: 0.9176 - val_precision_21: 0.0000e+00 - val_recall_21: 0.0000e+00
Epoch 2/16
43/43 [==============================] - 0s 1ms/step - loss: 0.1127 - accuracy: 0.8676 - precision_21: 0.0000e+00 - recall_21: 0.0000e+00 - val_loss: 0.0780 - val_accuracy: 0.9176 - val_precision_21: 0.0000e+00 - val_recall_21: 0.0000e+00
Epoch 3/16
43/43 [==============================] - 0s 2ms/step - loss: 0.0996 - accuracy: 0.8618 - precision_21: 0.4000 - recall_21: 0.0889 - val_loss: 0.0806 - val_accuracy: 0.9176 - val_precision_21: 0.0000e+00 - val_recall_21: 0.0000e+00
Epoch 4/16
43/43 [==============================] - 0s 2ms/step - loss: 0.0885 - accuracy: 0.8765 - precision_21: 0.5882 - recall_21: 0.2222 - val_loss: 0.0740 - val_accuracy: 0.9176 - val_precision_21: 0.0000e+00 - val_recall_21: 0.0000e+00
Epoch 5/16
43/43

fully connected neural network

In [143]:
# Defining model
inputs = keras.Input(shape=(6,1))
x = layers.BatchNormalization()(inputs)
x = layers.Dense(300, activation='relu')(x)
x = layers.Dense(150, activation='relu')(x)
x = layers.Dense(70, activation='relu')(x)
x = layers.Dropout(0.2)(x)
x = layers.Flatten()(x)
outputs = keras.layers.Dense(1, activation='sigmoid')(x)

model = keras.Model(inputs=inputs, outputs=outputs, name="classification-model")
model.summary()

Model: "classification-model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_26 (InputLayer)       [(None, 6, 1)]            0         
                                                                 
 batch_normalization_25 (Ba  (None, 6, 1)              4         
 tchNormalization)                                               
                                                                 
 dense_97 (Dense)            (None, 6, 300)            600       
                                                                 
 dense_98 (Dense)            (None, 6, 150)            45150     
                                                                 
 dense_99 (Dense)            (None, 6, 70)             10570     
                                                                 
 dropout_20 (Dropout)        (None, 6, 70)             0         
                                              

In [144]:
model.compile(
    loss=tf.keras.losses.MeanSquaredError(),
    optimizer=keras.optimizers.legacy.Adam(learning_rate=0.001),
    metrics=[ 'accuracy', tf.keras.metrics.Precision(),tf.keras.metrics.Recall()],
)

In [145]:
# Model training
ephs = 10

history = model.fit(X_train, np.array(y_train), batch_size=1 , epochs=ephs, validation_split=0.2)
test_scores_DNN_class = model.evaluate(X_test, np.array(y_test), verbose=1)


Epoch 1/10
340/340 [==============================] - 1s 2ms/step - loss: 0.1246 - accuracy: 0.8647 - precision_22: 0.0000e+00 - recall_22: 0.0000e+00 - val_loss: 0.0829 - val_accuracy: 0.9176 - val_precision_22: 0.0000e+00 - val_recall_22: 0.0000e+00
Epoch 2/10
340/340 [==============================] - 0s 1ms/step - loss: 0.1028 - accuracy: 0.8588 - precision_22: 0.0000e+00 - recall_22: 0.0000e+00 - val_loss: 0.0721 - val_accuracy: 0.9176 - val_precision_22: 0.0000e+00 - val_recall_22: 0.0000e+00
Epoch 3/10
340/340 [==============================] - 0s 919us/step - loss: 0.1027 - accuracy: 0.8618 - precision_22: 0.3750 - recall_22: 0.0667 - val_loss: 0.0811 - val_accuracy: 0.9059 - val_precision_22: 0.3333 - val_recall_22: 0.1429
Epoch 4/10
340/340 [==============================] - 0s 1ms/step - loss: 0.0920 - accuracy: 0.8647 - precision_22: 0.4737 - recall_22: 0.2000 - val_loss: 0.0789 - val_accuracy: 0.9059 - val_precision_22: 0.4000 - val_recall_22: 0.2857
Epoch 5/10
340/340 [==

### Classic ML models

In [146]:
# Testing classic ML algorithms 

from sklearn import svm
clf_SVC = svm.SVC()
clf_SVC.fit(X_train,y_train)

from sklearn.linear_model import SGDClassifier
clf_SGD = SGDClassifier(loss="hinge", penalty="l2", max_iter=5)
clf_SGD.fit(X_train, y_train)

from sklearn.neighbors import NearestCentroid
clf_NC = NearestCentroid()
clf_NC.fit(X_train, y_train)

/Users/aliceion/techtask/lib/python3.11/site-packages/sklearn/linear_model/_stochastic_gradient.py:713: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(


NearestCentroid()

In [147]:
# method for calculating precision and recall for received data
def metrics_class(y_test, y_pred):
    TN, TP, FN, FP = 0, 0, 0, 0

    for i in range(len(y_test)):
        if y_pred[i] == 0 and y_test[i] == 0:
            TN += 1
        if y_pred[i] == 1 and y_test[i] == 1:
            TP += 1
        if y_pred[i] == 0 and y_test[i] == 1:
            FN += 1
        if y_pred[i] == 1 and y_test[i] == 0:
            FP += 1
    
    precision = TP/(TP+FP)
    recall = TP/(TP+FN)
    
    return precision, recall

In [148]:
# Making predictions using the resulting models 
predicted_y_SVC = clf_SVC.predict(X_test)
predicted_y_SGD = clf_SGD.predict(X_test)
predicted_y_NC = clf_NC.predict(X_test)

precision_SVC, recall_SVC = metrics_class(y_test, predicted_y_SVC)
precision_SGD, recall_SGD = metrics_class(y_test, predicted_y_SGD)
precision_NC, recall_NC = metrics_class(y_test, predicted_y_NC)

In [150]:
# Receiving model F1-metric
F1_SVC = 2 * (precision_SVC * recall_SVC) / (precision_SVC + recall_SVC)
F1_SVD = 2 * (precision_SGD * recall_SGD) / (precision_SGD + recall_SGD)
F1_NC = 2 * (precision_NC * recall_NC) / (precision_NC + recall_NC)
F1_CNN = 2 * (test_scores_CNN_class[2] * test_scores_CNN_class[3]) / (test_scores_CNN_class[2] + test_scores_CNN_class[3])
F1_DNN = 2 * (test_scores_DNN_class[2] * test_scores_DNN_class[3]) / (test_scores_DNN_class[2] + test_scores_DNN_class[3])

### Classification results

In [152]:
# Returns summary table for gotten model metrics

class_summary = pd.DataFrame({'SVC':[precision_SVC, recall_SVC, F1_SVC],
                            'SGD':[precision_SGD, recall_SGD, F1_SVD],
                            'NC':[precision_NC, recall_NC, F1_NC],
                            'DNN':[test_scores_DNN_class[2], test_scores_DNN_class[3], F1_DNN],
                            'CNN':[test_scores_CNN_class[2], test_scores_CNN_class[3], F1_CNN]},
                           index=['precision', 'recall', 'F1']).T
class_summary

,precision,recall,F1
SVC,1.000000,0.066667,0.125000
SGD,0.666667,0.133333,0.222222
NC,0.187500,0.400000,0.255319
DNN,1.000000,0.200000,0.333333
CNN,0.846154,0.733333,0.785714


Вывод: для классификации предлагается использовать модели глубокого обучения, 
т.к. они показывают лучшие результаты. В частности, сверточную нейронную сеть.
Каждую полученную модель можно сохранить и использовать в дальнейшем.